<a href="https://colab.research.google.com/github/vatsalnanawati/UC_Davis_MSBA_Vatsal/blob/main/Building_a_Custom_Chatbot_with_Hugging_Face_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 815.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
test_val = dataset['test'].train_test_split(test_size=0.5, seed=42)

train_data = dataset['train']
val_data = test_val['train']
test_data = test_val['test']

print(f"Train: {len(train_data)}, Validation: {len(val_data)}, Test: {len(test_data)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Train: 1811, Validation: 226, Test: 227


In [ ]:
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding='max_length', max_length=128)

train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

train_data = train_data.rename_column('label', 'labels')
val_data = val_data.rename_column('label', 'labels')
test_data = test_data.rename_column('label', 'labels')

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/1811 [00:00<?, ? examples/s]

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./finbert-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-9-7185acc18b44>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akarrahe (akarrahe-university-of-california-davis) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.313716,0.880531
2,No log,0.207220,0.933628
3,No log,0.399087,0.933628
4,No log,0.418266,0.938053
5,0.247200,0.385689,0.942478


TrainOutput(global_step=570, training_loss=0.21721149358833045, metrics={'train_runtime': 328.1185, 'train_samples_per_second': 27.597, 'train_steps_per_second': 1.737, 'total_flos': 595622999381760.0, 'train_loss': 0.21721149358833045, 'epoch': 5.0})

In [ ]:
test_results = trainer.evaluate(test_data)
print(f"Test Set Accuracy: {test_results['eval_accuracy']:.4f}")


Test Set Accuracy: 0.9427


In [ ]:
class FinanceChatbot:
    def __init__(self):
        self.history = []
        self.label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

    def classify(self, sentence):
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_class = torch.argmax(logits, dim=-1).item()
        return self.label_map[predicted_class]

    def chat(self, user_message):
        sentiment = self.classify(user_message)
        self.history.append({"user": user_message, "bot": sentiment})
        return f"The sentiment is {sentiment}."


In [ ]:
def compute_bleu_for_chatbot(model, bot, raw_dataset, num_samples=100):
    model.eval()
    preds, refs = [], []

    label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

    for i, example in enumerate(raw_dataset):
        if i >= num_samples:
            break

        sentence = example['sentence']
        label = example['labels']

        predicted_sentiment = bot.classify(sentence)

        predicted_response = f"The sentiment is {predicted_sentiment}."
        reference_response = f"The sentiment is {label_map[label]}."

        preds.append(predicted_response)
        refs.append([reference_response])

        if i < 3:
            print(f"\nSample {i+1}:")
            print(f"Sentence: {sentence}")
            print(f"Generated: {predicted_response}")
            print(f"Reference: {reference_response}")

    return bleu.compute(predictions=preds, references=refs)

# Create bot outside, pass it in
bot = FinanceChatbot()

# Use original data (not tokenized)
bleu_score = compute_bleu_for_chatbot(model, bot, val_data.with_format("python"))
print(f"\nBLEU Score: {bleu_score['bleu']:.4f}")



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Sample 1:
Sentence: Oriola-KD is a spin-off of Finnish pharmaceutical group Orion Oyj 's wholesale division .
Generated: The sentiment is neutral.
Reference: The sentiment is neutral.

Sample 2:
Sentence: It posted a turnover of 4.5 mln euro $ 6.1 mln and an operating profit of 1.5 mln euro $ 2.0 mln for 2006 .
Generated: The sentiment is neutral.
Reference: The sentiment is neutral.

Sample 3:
Sentence: We succeeded in increasing our market share of sold apartment '' , comments Mr Kari Kauniskangas , Head of YIT International Construction Services .
Generated: The sentiment is positive.
Reference: The sentiment is positive.

BLEU Score: 0.9643


In [ ]:
# Step 1: Create the chatbot instance
bot = FinanceChatbot()

# Step 2: Start chatting!
response = bot.chat("Apple reports record quarterly loss.")
print(response)

response = bot.chat("Apple doesnt report.")
print(response)

response = bot.chat("Tesla faces regulatory investigation over self-driving crashes.")
print(response)

response = bot.chat("Tesla is good car company.")
print(response)

response = bot.chat("Avi's company reported profit of 17 trillion.")
print(response)

response = bot.chat("Avi reports record quarterly profitof 17 trillion.")
print(response)

# Step 3: Check full conversation history (optional)
print("\nConversation History:")
for turn in bot.history:
    print(f"User: {turn['user']}")
    print(f"Bot: {turn['bot']}")


The sentiment is negative.
The sentiment is neutral.
The sentiment is negative.
The sentiment is neutral.
The sentiment is neutral.
The sentiment is positive.

Conversation History:
User: Apple reports record quarterly loss.
Bot: negative
User: Apple doesnt report.
Bot: neutral
User: Tesla faces regulatory investigation over self-driving crashes.
Bot: negative
User: Tesla is good car company.
Bot: neutral
User: Avi's company reported profit of 17 trillion.
Bot: neutral
User: Avi reports record quarterly profitof 17 trillion.
Bot: positive
